In [1]:
#importing packages
import pretty_midi
import numpy as np
import mir_eval.display
import librosa.display
import matplotlib.pyplot as plt
%matplotlib inline
import IPython.display
import os
from mido import MidiFile

In [2]:
# Contains information for each individual note
class Note:
    def __init__(self, velocity, beginTick, endTick):
        self.velocity = velocity
        self.beginTick = beginTick
        self.endTick = endTick
    
    def addEndTick(self, endTick):
        self.endTick = endTick

In [3]:
# Contians information for each file
class File:
    def __init__(self, ticksPerSixteenthNote, style, tempo, isBeat, timeSignature, instruments):
        self.ticksPerSixteenthNote = ticksPerSixteenthNote
        self.style = style
        self.tempo = tempo
        self.isBeat = isBeat
        self.timeSignature = timeSignature
        self.instruments = instruments

In [4]:
# Contains information for each drummer (TODO: add functions)
class Drummer:
    def __init__(self, name, files):
        self.name = name
        self.files = files

In [5]:
def extractMidiData(filename):
    # Extract information from filename format
    filenameInfo = filename.split("_")
    style = filenameInfo[1]
    tempo = filenameInfo[2]
    # Files are classified as either beats or fills
    isBeat = (filenameInfo[3] == 'beat')
    # The split function keeps the file extension, so we have to get rid of it for the time signature
    timeSignature = filenameInfo[4].replace('.mid', '')
    
    # Extract information from Mido
    mid = MidiFile(filename)   
    ticksPerBeat = mid.ticks_per_beat
    # We will be checking to see how far off every note is from 16th note quantization
    ticksPerSixteenthNote = ticksPerBeat / 4
    
    # There should only be one track per file
    for track in mid.tracks:
        # Keeps track of what time we're at in the MIDI file using the "time" attribute
        # in "note_on" and "note_off" messages
        currentTick = 0
        
        # Dictionary for all of our MIDI instruments (snare, hi-hat, etc.)
        instruments = {}
        
        for msg in track:
            if msg.type == "note_on":
                if msg.note not in instruments:
                    # Create new list of notes for new instrument
                    instruments[msg.note] = []
                    
                currentTick += msg.time
                # End tick will be added when a note_off message appears
                instruments[msg.note].append(Note(msg.velocity, currentTick, 0))
            elif msg.type == "note_off":
                currentTick += msg.time
                # Add end tick value to the last note in the list
                lastNoteIndex = len(instruments[msg.note]) - 1
                instruments[msg.note][lastNoteIndex].addEndTick(currentTick)
    
    
                
    return File(ticksPerSixteenthNote, style, tempo, isBeat, timeSignature, instruments)

In [6]:
drummers = os.listdir('./groove')
drummersList = []

for drummer in drummers:
    if drummer.find('drummer') != -1:
        filesList = []
        sessions = os.listdir('./groove/' + drummer)
        for session in sessions:
            if session != '.DS_Store' and session != 'Icon\r':
                files = os.listdir('./groove/' + drummer + '/' + session)
                for file in files:
                    if file.endswith('.mid'):
                        filePath = './groove/' + drummer + "/" + session + "/"  + file
                        filesList.append(extractMidiData(filePath))

        drummersList.append(Drummer(drummer, filesList))